In [4]:
import digitalhub as dh
import pandas as pd
import requests
import os

In [5]:
PROJECT = "demo-etl-nk"
project = dh.get_or_create_project(PROJECT)

In [12]:
print(project)

{'kind': 'project', 'metadata': {'project': 'demo-etl-nk', 'name': 'demo-etl-nk', 'created': '2024-10-23T11:48:54.947Z', 'updated': '2024-10-23T11:48:54.947Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'context': './', 'functions': [], 'artifacts': [], 'workflows': [], 'dataitems': [], 'models': []}, 'status': {'state': 'CREATED'}, 'user': 'khurshid@fbk.eu', 'id': 'demo-etl-nk', 'name': 'demo-etl-nk', 'key': 'store://demo-etl-nk'}


In [4]:
URL = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/rilevazione-flusso-veicoli-tramite-spire-anno-2023/exports/csv?lang=it&timezone=Europe%2FRome&use_labels=true&delimiter=%3B"
filename = "rilevazione-flusso-veicoli-tramite-spire-anno-2023.csv"

In [5]:
with requests.get(URL) as r:
    with open(filename, "wb") as f:
        f.write(r.content)

In [6]:
df = pd.read_csv(filename, sep=";")

In [7]:
df

,data,codice spira,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,...,ordinanza,stato,codimpsem,direzione,angolo,longitudine,latitudine,geopoint,ID_univoco_stazione_spira,giorno settimana
0,2023-03-25,0.127 3.89 6 1,97,55,31,16,15,38,77,194,...,4000/343434,A,156,N,348.0,11.373242,44.499532,"44.4995320603747, 11.3732419756135",207.0,Sabato
1,2023-03-25,0.127 3.89 8 1,52,25,12,14,5,13,24,69,...,4000/343434,A,156,E,242.0,11.372867,44.499893,"44.4998932893995, 11.3728670312021",208.0,Sabato
2,2023-03-25,0.127 3.90 6 1,167,94,69,37,42,62,85,151,...,4000/343434,A,232,NE,308.0,11.410369,44.524032,"44.52403243227343, 11.410369186317224",211.0,Sabato
3,2023-03-25,0.127 3.93 4 1,5,3,0,5,5,7,10,26,...,4000/343434,A,106,SE,217.0,11.381753,44.518132,"44.5181320170998, 11.381752851058",217.0,Sabato
4,2023-03-25,0.127 3.93 6 1,234,136,95,85,55,103,141,287,...,4000/343434,A,106,NE,321.0,11.381681,44.517642,"44.5176422952597, 11.3816808772175",218.0,Sabato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313062,2023-11-24,0.127 3.56 2 2,7,5,1,3,3,4,14,71,...,4000/343434,A,276,E,256.0,11.340309,44.508985,"44.5089851213122, 11.3403094159011",174.0,Venerdì
313063,2023-11-24,0.127 3.56 4 1,83,31,22,12,43,74,199,725,...,4000/343434,A,276,S,172.0,11.340593,44.509202,"44.5092024956841, 11.3405934767054",175.0,Venerdì
313064,2023-11-24,0.127 3.59 4 1,63,19,21,10,22,44,121,515,...,4000/343434,A,397,NaN,81.0,11.337767,44.508073,"44.5080725926701, 11.3377666997372",176.0,Venerdì
313065,2023-11-24,0.127 3.64 6 1,190,78,34,48,68,159,305,825,...,4000/343434,A,372,N,351.0,11.328969,44.519481,"44.5194807065927, 11.3289690749773",180.0,Venerdì


In [13]:
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [14]:
%%writefile "src/download-data.py"

from digitalhub_runtime_python import handler

@handler(outputs=["dataset"])
def downloader(url):
    # read and rewrite to normalize and export as data
    df = url.as_df(file_format='csv',sep=";")
    return df

Overwriting src/download-data.py


In [17]:
func = project.new_function(
                         name="download-data",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/download-data.py",
                         handler="downloader")

In [18]:
URL = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/rilevazione-flusso-veicoli-tramite-spire-anno-2023/exports/csv?lang=it&timezone=Europe%2FRome&use_labels=true&delimiter=%3B"
di= project.new_dataitem(name="url_data_item",kind="table",path=URL)

In [19]:
run = func.run(action="job", inputs={'url':di.key}, outputs={"dataset": "dataset"}, local_execution=False)

In [9]:
dataset_di = project.get_dataitem('dataset')

In [19]:
dataset_df = dataset_di.as_df()

In [20]:
dataset_df

,data,codice spira,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,...,ordinanza,stato,codimpsem,direzione,angolo,longitudine,latitudine,geopoint,ID_univoco_stazione_spira,giorno settimana
0,2023-03-25,0.127 3.89 6 1,97,55,31,16,15,38,77,194,...,4000/343434,A,156,N,348.0,11.373242,44.499532,"44.4995320603747, 11.3732419756135",207.0,Sabato
1,2023-03-25,0.127 3.89 8 1,52,25,12,14,5,13,24,69,...,4000/343434,A,156,E,242.0,11.372867,44.499893,"44.4998932893995, 11.3728670312021",208.0,Sabato
2,2023-03-25,0.127 3.90 6 1,167,94,69,37,42,62,85,151,...,4000/343434,A,232,NE,308.0,11.410369,44.524032,"44.52403243227343, 11.410369186317224",211.0,Sabato
3,2023-03-25,0.127 3.93 4 1,5,3,0,5,5,7,10,26,...,4000/343434,A,106,SE,217.0,11.381753,44.518132,"44.5181320170998, 11.381752851058",217.0,Sabato
4,2023-03-25,0.127 3.93 6 1,234,136,95,85,55,103,141,287,...,4000/343434,A,106,NE,321.0,11.381681,44.517642,"44.5176422952597, 11.3816808772175",218.0,Sabato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313062,2023-11-24,0.127 3.56 2 2,7,5,1,3,3,4,14,71,...,4000/343434,A,276,E,256.0,11.340309,44.508985,"44.5089851213122, 11.3403094159011",174.0,Venerdì
313063,2023-11-24,0.127 3.56 4 1,83,31,22,12,43,74,199,725,...,4000/343434,A,276,S,172.0,11.340593,44.509202,"44.5092024956841, 11.3405934767054",175.0,Venerdì
313064,2023-11-24,0.127 3.59 4 1,63,19,21,10,22,44,121,515,...,4000/343434,A,397,None,81.0,11.337767,44.508073,"44.5080725926701, 11.3377666997372",176.0,Venerdì
313065,2023-11-24,0.127 3.64 6 1,190,78,34,48,68,159,305,825,...,4000/343434,A,372,N,351.0,11.328969,44.519481,"44.5194807065927, 11.3289690749773",180.0,Venerdì


In [21]:
%%writefile "src/process-spire.py"

from digitalhub_runtime_python import handler

KEYS=['codice spira','longitudine','latitudine','Livello','tipologia','codice','codice arco','codice via','Nome via', 'stato','direzione','angolo','geopoint']

@handler(outputs=["dataset-spire"])
def process(project, di):
    df = di.as_df()
    sdf= df.groupby(['codice spira']).first().reset_index()[KEYS]
    return sdf

Writing src/process-spire.py


In [22]:
process_func = project.new_function(
                         name="process-spire",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/process-spire.py",
                         handler="process")

In [23]:
process_run = process_func.run(action="job", inputs={'di': dataset_di.key}, outputs={'dataset-spire': 'dataset-spire'}, local_execution=False)

2024-10-23 11:54:03,318 - INFO - Validating task.
2024-10-23 11:54:03,319 - INFO - Validating run.
2024-10-23 11:54:03,320 - INFO - Starting task.
2024-10-23 11:54:03,320 - INFO - Configuring execution.
2024-10-23 11:54:03,322 - INFO - Composing function arguments.
2024-10-23 11:54:03,399 - INFO - Executing run.
2024-10-23 11:54:03,821 - INFO - Task completed, returning run status.


In [24]:
spire_di = project.get_dataitem('dataset-spire')
spire_df = spire_di.as_df()

In [25]:
spire_df

,codice spira,longitudine,latitudine,Livello,tipologia,codice,codice arco,codice via,Nome via,stato,direzione,angolo,geopoint
0,0.127 1.1 6 1,11.354166,44.498535,1,spira,498.0,3312,19900,VIA G.BATTISTA DE ROLANDIS,A,N,342.0,"44.4985349106485, 11.3541657967424"
1,0.127 1.12 8 1,11.338970,44.495251,1,spira,1045.0,1016,5900,VIA CESARE BATTISTI,A,N,350.0,"44.4952505129043, 11.338970003537"
2,0.127 1.13 6 1,11.346420,44.491648,1,spira,130.0,1169,14700,VIA CASTIGLIONE,A,S,198.0,"44.4916483847646, 11.3464200565732"
3,0.127 1.14 4 1,11.339836,44.490116,1,spira,521.0,1050,59900,VIA URBANA,A,E,264.0,"44.4901162203284, 11.3398356513878"
4,0.127 1.15 2 1,11.343358,44.489507,1,spira,132.0,1064,25800,VIA GARIBALDI,A,N,347.0,"44.4895074220971, 11.3433581064329"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,4.98 0.127 4 1,11.385065,44.469348,1,spira,946.0,1989,15350,VIA FRANCESCO CAVAZZONI,A,NO,69.0,"44.4693478190501, 11.3850649221279"
901,4.98 0.127 6 1,11.385481,44.469397,1,spira,899.0,2002,23230,VIA FIRENZE,A,N,352.0,"44.4693967343944, 11.385480982166"
902,4.98 0.127 8 1,11.385614,44.469066,1,spira,912.0,1960,42780,VIA OSOPPO,A,E,257.0,"44.469065830958, 11.3856143249703"
903,4.98 4.98 3 1,11.385260,44.468970,1,spira,1067.0,1963,32510,VIA LUIGI LONGO,A,S,170.0,"44.4689700816957, 11.3852604232066"


In [6]:
%%writefile "src/process-measures.py"

from digitalhub_runtime_python import handler
import pandas as pd

KEYS = ['00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00', '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00', '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00', '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00', '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-24:00']
COLUMNS=['data','codice spira']

@handler(outputs=["dataset-measures"])
def process(project, di):
    df = di.as_df()
    rdf = df[COLUMNS+KEYS]
    ls = []
    for key in KEYS:
        k = key.split("-")[0]
        xdf = rdf[COLUMNS + [key]]
        xdf['time'] = xdf.data.apply(lambda x: x+' ' +k)
        xdf['value'] = xdf[key]
        ls.append(xdf[['time','codice spira','value']])
    edf = pd.concat(ls)
    return edf

Overwriting src/process-measures.py


In [7]:
process_measures_func = project.new_function(
                         name="process-measures",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/process-measures.py",
                         handler="process")

In [10]:
process_measures_run = process_measures_func.run(action="job", inputs={'di': dataset_di.key}, outputs={'dataset-measures': 'dataset-measures'}, local_execution=False)

In [11]:
measures_di = project.get_dataitem('dataset-measures')
measures_df = measures_di.as_df()
measures_df.head()

,time,codice spira,value
0,2023-03-25 00:00,0.127 3.89 6 1,97
1,2023-03-25 00:00,0.127 3.89 8 1,52
2,2023-03-25 00:00,0.127 3.90 6 1,167
3,2023-03-25 00:00,0.127 3.93 4 1,5
4,2023-03-25 00:00,0.127 3.93 6 1,234


In [12]:
%%writefile "src/pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def pipeline(url):
    with pipeline_context() as pc:
        downloader = pc.step(
            name="download-data",
            function="download-data",
            action="job",
            inputs={"url": url},
            outputs={"dataset": "dataset"},
        )

        process_spire = pc.step(
            name="process-spire",
            function="process-spire",
            action="job",
            inputs={"di": downloader.outputs["dataset"]}
        )

        process_measures = pc.step(
            name="process-measures",
            function="process-measures",
            action="job",
            inputs={"di": downloader.outputs["dataset"]}
        )

Writing src/pipeline.py


In [13]:
workflow = project.new_workflow(name="pipeline", kind="kfp", code_src="src/pipeline.py", handler="pipeline")

In [20]:
workflow.run(parameters={"url": di.key})

{'kind': 'kfp+run', 'metadata': {'project': 'demo-etl-nk', 'name': 'd96a8bd1-20cc-4121-8d97-cb61af6ab18e', 'created': '2024-10-23T12:07:44.118Z', 'updated': '2024-10-23T12:07:44.132Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'kfp+pipeline://demo-etl-nk/pipeline:b24cf917-fbbc-4557-bcc8-6b821870b16c', 'local_execution': False, 'function': 'kfp://demo-etl-nk/pipeline:b24cf917-fbbc-4557-bcc8-6b821870b16c', 'source': {'source': 'src/pipeline.py', 'handler': 'pipeline', 'base64': 'CmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX2tmcC5kc2wgaW1wb3J0IHBpcGVsaW5lX2NvbnRleHQKCmRlZiBwaXBlbGluZSh1cmwpOgogICAgd2l0aCBwaXBlbGluZV9jb250ZXh0KCkgYXMgcGM6CiAgICAgICAgZG93bmxvYWRlciA9IHBjLnN0ZXAoCiAgICAgICAgICAgIG5hbWU9ImRvd25sb2FkLWRhdGEiLAogICAgICAgICAgICBmdW5jdGlvbj0iZG93bmxvYWQtZGF0YSIsCiAgICAgICAgICAgIGFjdGlvbj0iam9iIiwKICAgICAgICAgICAgaW5wdXRzPXsidXJsIjogdXJsfSwKICAgICAgICAgICAgb3V0cHV0cz17ImRhdGFzZXQiOiAiZGF0YXNldCJ9LAogICAgICAgICkKCiAgICAgICAgcHJvY2Vzc19zcGlyZSA9IHBjLnN0

In [21]:
%%writefile 'src/api.py'

def init_context(context):
    di = context.project.get_dataitem('dataset-measures')
    df = di.as_df()
    setattr(context, "df", df)

def handler(context, event):
    df = context.df

    if df is None:
        return ""

    # mock REST api
    method = event.method
    path = event.path
    fields = event.fields

    id = False

    # pagination
    page = 0
    pageSize = 50

    if "page" in fields:
        page = int(fields['page'])

    if "size" in fields:
        pageSize = int(fields['size'])

    if page < 0:
        page = 0

    if pageSize < 1:
        pageSize = 1

    if pageSize > 100:
        pageSize = 100

    start = page * pageSize
    end = start + pageSize
    total = len(df)

    if end > total:
        end = total

    ds = df.iloc[start:end]
    json = ds.to_json(orient="records")

    res = {"data": json, "page": page, "size": pageSize, "total": total}

    return res

Writing src/api.py


In [22]:
api_func = project.new_function(
                         name="api",
                         kind="python",
                         python_version="PYTHON3_10",
                         code_src="src/api.py",
                         handler="handler",
                         init_function="init_context")

In [23]:
run_serve_model = api_func.run(action="serve")

In [26]:
run_serve_model.refresh()

{'kind': 'python+run', 'metadata': {'project': 'demo-etl-nk', 'name': 'c4029413-e113-4ed7-97d5-787e585ee5a8', 'created': '2024-10-23T12:13:02.339Z', 'updated': '2024-10-23T12:13:28.069Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'python+serve://demo-etl-nk/api:fa0ef681-78ed-410d-b47b-ff23cf0bce25', 'local_execution': False, 'function': 'python://demo-etl-nk/api:fa0ef681-78ed-410d-b47b-ff23cf0bce25', 'source': {'source': 'src/api.py', 'handler': 'handler', 'base64': 'CmRlZiBpbml0X2NvbnRleHQoY29udGV4dCk6CiAgICBkaSA9IGNvbnRleHQucHJvamVjdC5nZXRfZGF0YWl0ZW0oJ2RhdGFzZXQtbWVhc3VyZXMnKQogICAgZGYgPSBkaS5hc19kZigpCiAgICBzZXRhdHRyKGNvbnRleHQsICJkZiIsIGRmKQoKZGVmIGhhbmRsZXIoY29udGV4dCwgZXZlbnQpOgogICAgZGYgPSBjb250ZXh0LmRmCgogICAgaWYgZGYgaXMgTm9uZToKICAgICAgICByZXR1cm4gIiIKCiAgICAjIG1vY2sgUkVTVCBhcGkKICAgIG1ldGhvZCA9IGV2ZW50Lm1ldGhvZAogICAgcGF0aCA9IGV2ZW50LnBhdGgKICAgIGZpZWxkcyA9IGV2ZW50LmZpZWxkcwoKICAgIGlkID0gRmFsc2UKCiAgICAjIHBhZ2luYXRpb24KICAgIHBhZ2UgPS

In [29]:
SERVICE_URL = f"http://{run_serve_model.status.to_dict()['service']['url']}"
SERVICE_URL

'http://s-pythonserve-c4029413-e113-4ed7-97d5-787e585ee5a8.digitalhub-tenant1:8080'

In [30]:
with requests.get(f'{SERVICE_URL}/?page=5&size=10') as r:
    res = r.json()
print(res)

{'data': '[{"time":"2023-03-25 00:00","codice spira":"1.8 1.11 6 1","value":100},{"time":"2023-03-25 00:00","codice spira":"1.8 1.12 2 1","value":140},{"time":"2023-03-25 00:00","codice spira":"1.8 1.18 6 1","value":106},{"time":"2023-03-25 00:00","codice spira":"1.9 1.8 2 1","value":146},{"time":"2023-03-25 00:00","codice spira":"1.11 1.8 2 1","value":155},{"time":"2023-03-25 00:00","codice spira":"1.12 1.8 6 1","value":98},{"time":"2023-03-25 00:00","codice spira":"1.13 1.19 8 1","value":0},{"time":"2023-03-25 00:00","codice spira":"1.16 1.17 8 1","value":28},{"time":"2023-03-25 00:00","codice spira":"1.17 4.19 6 1","value":151},{"time":"2023-03-25 00:00","codice spira":"1.18 1.10 4 1","value":78}]', 'page': 5, 'size': 10, 'total': 7513608}


In [31]:
rdf = pd.read_json(res['data'], orient='records')
rdf.head()

,time,codice spira,value
0,2023-03-25 00:00,1.8 1.11 6 1,100
1,2023-03-25 00:00,1.8 1.12 2 1,140
2,2023-03-25 00:00,1.8 1.18 6 1,106
3,2023-03-25 00:00,1.9 1.8 2 1,146
4,2023-03-25 00:00,1.11 1.8 2 1,155
